First layer info scraped into a pickle file, total 180 links for layer 2 property details scraping.
Info will be updated to the existing pickle.

Encountered obstables
- timeout with only 138 results
- if only 138 properties info are available, make sure no skipping and info fall into another listing

In [87]:
# open the pickle file that scraped with relevant property information as objects
import pickle

objects = []
with (open("\YOUR\PICKLE\FILE\ADDRESS", "rb")) as openfile: #copy the pickle file path here
    while True:
        try:
            objects.append(pickle.load(openfile))
        except EOFError:
            break

In [88]:
# loop the pickle file (consists of one big data list storing data_tuple dictionaries) back to a list call object_list
# check number of links and view the first 5 addresses
# info should have source links, address and many None items

objects_list = []
for object in objects:
    objects_list.extend(object)
    print(len(objects_list))
    print(objects_list[0:5])

180
[{'id': 'Z14800172023', 'source': 'http://cppcl.property.hk/tran_prop_detail.php?ref=Z1480017&year=2023', 'chinese': {'raw': None, 'address': '北角 建華街57號', 'floor': '4', 'flat': ''}, 'english': {'raw': None, 'address': None, 'floor': None, 'flat': None}, 'data': {'registration_number': None, 'registration_date': None, 'docs_date': None, 'gross_floor_area': '', 'saleable_floor_area': None, 'price(million)': '190.000', 'price_per_feet_gross': '', 'price_per_feet_saleable': None, 'district': None, 'usage': None, 'occupation_permit': None, 'lease_term': None, 'renewable_term': None, 'facing': None, 'layout': None, 'seller': None, 'buyer': None, 'remark': None}}, {'id': '078928392023', 'source': 'http://cppcl.property.hk/tran_prop_detail.php?ref=07892839&year=2023', 'chinese': {'raw': None, 'address': '北角 百福花園 百佳閣 C座', 'floor': '14', 'flat': 'E'}, 'english': {'raw': None, 'address': None, 'floor': None, 'flat': None}, 'data': {'registration_number': None, 'registration_date': None, 'docs

In [71]:
# extract the source links from objects_list, check total number of links

links=[]

for properties in objects_list:
    links.append(properties['source'])

print(len(links))
# print(links)

180


In [72]:
from typing import Tuple
from time import sleep

In [73]:
import spacy
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
import pandas as pd
import re
print(spacy.__version__)

3.3.1


In [74]:
# call chrome drive
browser = webdriver.Chrome(service=Service("YOUR\PATH.exe")) #copy your chrome driver address here

In [75]:
# to work around the timeout problem, the source links are scraped in batches
links_batch1 = links[0:50]
links_batch2 = links[50:100]
links_batch3 = links[100:180]
print(links_batch3)

['http://cppcl.property.hk/tran_prop_detail.php?ref=07892748&year=2023', 'http://cppcl.property.hk/tran_prop_detail.php?ref=07892750&year=2023', 'http://cppcl.property.hk/tran_prop_detail.php?ref=07893338&year=2023', 'http://cppcl.property.hk/tran_prop_detail.php?ref=07893054&year=2023', 'http://cppcl.property.hk/tran_prop_detail.php?ref=Z1300103&year=2023', 'http://cppcl.property.hk/tran_prop_detail.php?ref=07893055&year=2023', 'http://cppcl.property.hk/tran_prop_detail.php?ref=07893307&year=2023', 'http://cppcl.property.hk/tran_prop_detail.php?ref=07893046&year=2023', 'http://cppcl.property.hk/tran_prop_detail.php?ref=07893500&year=2023', 'http://cppcl.property.hk/tran_prop_detail.php?ref=07893531&year=2023', 'http://cppcl.property.hk/tran_prop_detail.php?ref=07893532&year=2023', 'http://cppcl.property.hk/tran_prop_detail.php?ref=07893529&year=2023', 'http://cppcl.property.hk/tran_prop_detail.php?ref=07893530&year=2023', 'http://cppcl.property.hk/tran_prop_detail.php?ref=07893535&yea

In [76]:
# gather the XPATHs for the target info we need

xpath_address = '/html/body/table/tbody/tr/td/table/tbody/tr[3]/td[2]'
xpath_registration_number = '/html/body/table/tbody/tr[1]/td/table/tbody/tr[1]/td[4]'
xpath_registration_date = '/html/body/table/tbody/tr[1]/td/table/tbody/tr[1]/td[2]'
xpath_registration_nature = '/html/body/table/tbody/tr[1]/td/table/tbody/tr[1]/td[6]'
xpath_docs_date = '/html/body/table/tbody/tr[1]/td/table/tbody/tr[2]/td[2]'
xpath_gross_area = '/html/body/table/tbody/tr[1]/td/table/tbody/tr[5]/td[2]'
xpath_price_per_feet_gross = '/html/body/table/tbody/tr[1]/td/table/tbody/tr[6]/td[2]'
xpath_price_million = '/html/body/table/tbody/tr/td/table/tbody/tr[4]/td[2]'
xpath_district = '/html/body/table/tbody/tr[1]/td/table/tbody/tr[2]/td[4]'
xpath_usage = '/html/body/table/tbody/tr[1]/td/table/tbody/tr[2]/td[6]'
xpath_occupation_permit = '/html/body/table/tbody/tr[1]/td/table/tbody/tr[4]/td[4]'
xpath_lease_term = '/html/body/table/tbody/tr[1]/td/table/tbody/tr[5]/td[4]'
xpath_renewable_term = '/html/body/table/tbody/tr[1]/td/table/tbody/tr[6]/td[4]'
xpath_facing = '/html/body/table/tbody/tr[1]/td/table/tbody/tr[7]/td[2]'
xpath_layout = '/html/body/table/tbody/tr[1]/td/table/tbody/tr[7]/td[4]'
xpath_seller = '/html/body/table/tbody/tr[1]/td/table/tbody/tr[8]/td[2]'
xpath_buyer = '/html/body/table/tbody/tr[1]/td/table/tbody/tr[9]/td[2]'
xpath_remarks = '/html/body/table/tbody/tr[1]/td/table/tbody/tr[10]/td[2]'


In [77]:
# only None items are updated with new info
def update_dict(objects_list, browser, xpath, key1, key2):
    if objects_list[key1][key2] is None:
        objects_list[key1][key2] = list(map(lambda x: x.text, browser.find_elements("xpath", xpath)))

# looping the right batch of links to update the following items(if None) in objects_list
for link in links_batch3:  # come back and change this line to the right batch for scraping
    browser.get(link)
    for i, a in enumerate(objects_list):
        if 100 <= i <= 180: # come back and change this line to the right index
            if browser.current_url == objects_list[i]['source']:
                update_dict(objects_list[i], browser, xpath_address, 'chinese', 'address')
                update_dict(objects_list[i], browser, xpath_address, 'english', 'address')
                update_dict(objects_list[i], browser, xpath_registration_number, 'data', 'registration_number')
                update_dict(objects_list[i], browser, xpath_registration_date, 'data', 'registration_date')
                update_dict(objects_list[i], browser, xpath_docs_date, 'data', 'docs_date')
                update_dict(objects_list[i], browser, xpath_district, 'data', 'district')
                update_dict(objects_list[i], browser, xpath_occupation_permit, 'data', 'occupation_permit')
                update_dict(objects_list[i], browser, xpath_lease_term, 'data', 'lease_term')
                update_dict(objects_list[i], browser, xpath_renewable_term, 'data', 'renewable_term')
                update_dict(objects_list[i], browser, xpath_facing, 'data', 'facing')
                update_dict(objects_list[i], browser, xpath_layout, 'data', 'layout')
                update_dict(objects_list[i], browser, xpath_seller, 'data', 'seller')
                update_dict(objects_list[i], browser, xpath_buyer, 'data', 'buyer')
                update_dict(objects_list[i], browser, xpath_remarks, 'data', 'remark')


In [80]:
# checking if the newly scraped info fit in new data to objects_list as we wanted
print(objects_list[10]) #compare property index before and after scraping

{'id': '078934842023', 'source': 'http://cppcl.property.hk/tran_prop_detail.php?ref=07893484&year=2023', 'chinese': {'raw': None, 'address': '鰂魚涌 康怡花園 C座', 'floor': '2', 'flat': '8'}, 'english': {'raw': None, 'address': None, 'floor': None, 'flat': None}, 'data': {'registration_number': None, 'registration_date': None, 'docs_date': None, 'gross_floor_area': '752', 'saleable_floor_area': None, 'price(million)': '7.980', 'price_per_feet_gross': '@10,612', 'price_per_feet_saleable': None, 'district': None, 'usage': None, 'occupation_permit': None, 'lease_term': None, 'renewable_term': None, 'facing': None, 'layout': None, 'seller': None, 'buyer': None, 'remark': None}}


In [177]:
# save the updated objects_list to a new pickle file without affecting the raw one
# the the subsequent scraping of batch 2, batch 3 will update to ccpclFilled as well
import pickle
with open ('ccpclFilled.pickle', 'wb') as f:
    pickle.dump(objects_list, f)

In [178]:
# check the saved pickle file
import pickle

objectsFilled = []
with (open("ccpclFilled.pickle", "rb")) as openfile:
    while True:
        try:
            objectsFilled.append(pickle.load(openfile))
        except EOFError:
            break

print(objectsFilled)

[[{'id': 'Z14800172023', 'source': 'http://cppcl.property.hk/tran_prop_detail.php?ref=Z1480017&year=2023', 'chinese': {'raw': None, 'address': '北角 建華街57號', 'floor': '4', 'flat': ''}, 'english': {'raw': None, 'address': ['FLOOR 4, KIN WAH ST 57\n建華街57號, 4樓'], 'floor': None, 'flat': None}, 'data': {'registration_number': ['23013101480X1X'], 'registration_date': ['2023-01-31'], 'docs_date': ['2023-01-04'], 'gross_floor_area': '', 'saleable_floor_area': None, 'price(million)': '190.000', 'price_per_feet_gross': '', 'price_per_feet_saleable': None, 'district': ['北角'], 'usage': None, 'occupation_permit': [' '], 'lease_term': ['1997 (尚餘-26年)'], 'renewable_term': ['2072'], 'facing': [' '], 'layout': [' '], 'seller': [' '], 'buyer': [' '], 'remark': [' ']}}, {'id': '078928392023', 'source': 'http://cppcl.property.hk/tran_prop_detail.php?ref=07892839&year=2023', 'chinese': {'raw': None, 'address': '北角 百福花園 百佳閣 C座', 'floor': '14', 'flat': 'E'}, 'english': {'raw': None, 'address': ['UNIT E, FLOOR 

In [183]:
listFilled = []
for d in objectsFilled:
    listFilled.extend(d)
    print(listFilled[179:180])

[{'id': '078923062023', 'source': 'http://cppcl.property.hk/tran_prop_detail.php?ref=07892306&year=2023', 'chinese': {'raw': None, 'address': '沙田 碧濤花園 第03期 第02座 雅芝閣', 'floor': '8', 'flat': 'E'}, 'english': {'raw': None, 'address': ['UNIT E, FLOOR 8, PICTORIAL GDN PH 03 TWR 02 IRIS CT, ON KING ST 25\n安景街25號, 碧濤花園 第03期 第02座 雅芝閣, 8樓 E室'], 'floor': None, 'flat': None}, 'data': {'registration_number': ['23013000620X2X'], 'registration_date': ['2023-01-30'], 'docs_date': ['2023-01-20'], 'gross_floor_area': '1026', 'saleable_floor_area': None, 'price(million)': '8.700', 'price_per_feet_gross': '@8,480', 'price_per_feet_saleable': None, 'district': ['沙田'], 'usage': None, 'occupation_permit': ['1993'], 'lease_term': ['2047 (尚餘24年)'], 'renewable_term': [' '], 'facing': ['東南'], 'layout': ['3房(1套)2廳1貯'], 'seller': [' '], 'buyer': [' '], 'remark': ["樓底9.35', 實積包露台21', 另有窗台34', CPS NO.105"]}}]


In [24]:
import pandas as pd

ccpcl_dict = pd.read_pickle('ccpclFilled.pickle')
ccpcl_df = pd.json_normalize(ccpcl_dict)
ccpcl_dict
# ccpcl_chi = pd.DataFrame(ccpcl_dict['chinese'])

[{'id': 'Z14800172023',
  'source': 'http://cppcl.property.hk/tran_prop_detail.php?ref=Z1480017&year=2023',
  'chinese': {'raw': None, 'address': '北角 建華街57號', 'floor': '4', 'flat': ''},
  'english': {'raw': None,
   'address': ['FLOOR 4, KIN WAH ST 57\n建華街57號, 4樓'],
   'floor': None,
   'flat': None},
  'data': {'registration_number': ['23013101480X1X'],
   'registration_date': ['2023-01-31'],
   'docs_date': ['2023-01-04'],
   'gross_floor_area': '',
   'saleable_floor_area': None,
   'price(million)': '190.000',
   'price_per_feet_gross': '',
   'price_per_feet_saleable': None,
   'district': ['北角'],
   'usage': None,
   'occupation_permit': [' '],
   'lease_term': ['1997 (尚餘-26年)'],
   'renewable_term': ['2072'],
   'facing': [' '],
   'layout': [' '],
   'seller': [' '],
   'buyer': [' '],
   'remark': [' ']}},
 {'id': '078928392023',
  'source': 'http://cppcl.property.hk/tran_prop_detail.php?ref=07892839&year=2023',
  'chinese': {'raw': None,
   'address': '北角 百福花園 百佳閣 C座',
   'fl

In [44]:
from pandas import json_normalize
import pandas as pd

ccpcl_df = pd.json_normalize(ccpcl_dict)
ccpcl_df

,id,source,chinese.raw,chinese.address,chinese.floor,chinese.flat,english.raw,english.address,english.floor,english.flat,...,data.district,data.usage,data.occupation_permit,data.lease_term,data.renewable_term,data.facing,data.layout,data.seller,data.buyer,data.remark
0,Z14800172023,http://cppcl.property.hk/tran_prop_detail.php?...,None,北角 建華街57號,4,,None,"[FLOOR 4, KIN WAH ST 57\n建華街57號, 4樓]",None,None,...,[北角],None,[ ],[1997 (尚餘-26年)],[2072],[ ],[ ],[ ],[ ],[ ]
1,078928392023,http://cppcl.property.hk/tran_prop_detail.php?...,None,北角 百福花園 百佳閣 C座,14,E,None,"[UNIT E, FLOOR 14, BEDFORD GDN BLK C (PAK KAI ...",None,None,...,[北角],None,[1981],[2012 (尚餘-11年)],[ ],[北],[2房2廳],[ ],[ ],"[BAL,]"
2,078929782023,http://cppcl.property.hk/tran_prop_detail.php?...,None,北角 百福花園 百德閣 G座,16,C,None,"[UNIT C, FLOOR 16, BEDFORD GDN BLK G (PAK TAK ...",None,None,...,[北角],None,[1981],[2012 (尚餘-11年)],[ ],[南],[1房2廳],[ ],[ ],"[BAL,]"
3,078927322023,http://cppcl.property.hk/tran_prop_detail.php?...,None,北角 百福花園 百祥閣 H座,11,B,None,"[UNIT B, FLOOR 11, BEDFORD GDN BLK H (PAK CHEU...",None,None,...,[北角],None,[1981],[2012 (尚餘-11年)],[ ],[西],[2房2廳],[ ],[ ],"[BAL,]"
4,Z19000292023,http://cppcl.property.hk/tran_prop_detail.php?...,None,北角 永安苑,8,29,None,"[UNIT 29, FLOOR 8, ERNEST COURT, NGAN MOK ST 2...",None,None,...,[北角],None,[1964],[1997 (尚餘-26年)],[2072],[ ],[ ],[ ],[ ],[ ]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,078924872023,http://cppcl.property.hk/tran_prop_detail.php?...,None,沙田 希爾頓中心 C座,9,6,None,"[UNIT 6, FLOOR 9, HILTON PLAZA BLK C, SHATIN C...",None,None,...,[沙田],None,[1985],[2047 (尚餘24年)],[ ],[西北],[2房2廳],[ ],[ ],"[BW:57',]"
176,078923892023,http://cppcl.property.hk/tran_prop_detail.php?...,None,沙田 好運中心 竹林閣,16,B,None,"[UNIT B, FLOOR 16, LUCKY PLAZA CHUK LAM COURT ...",None,None,...,[沙田],None,[1983],[2047 (尚餘24年)],[ ],[西北],[2房2廳],[ ],[ ],"[BW:37',]"
177,078923902023,http://cppcl.property.hk/tran_prop_detail.php?...,None,沙田 御龍山 第06座,11,D,None,"[UNIT D, FLOOR 11, PALAZZO TWR 06, LOK KING ST...",None,None,...,[沙田],None,[2008],[2053 (尚餘30年)],[ ],[西北],[3房(1套)2廳1貯],[ ],[ ],"[HT:10'4"", 實積包露台22'UP16'窗台24', AR46',]"
178,078921062023,http://cppcl.property.hk/tran_prop_detail.php?...,None,沙田 御龍山 第10座,28,C,None,"[UNIT C, FLOOR 28, PALAZZO TWR 10, LOK KING ST...",None,None,...,[沙田],None,[2008],[2053 (尚餘30年)],[ ],[西],[3房(1套)2廳1貯],[ ],[ ],"[HT:10'4"", 實積包露台22'UP16'窗台28', AR46',]"
